# FIRST YEAR BIRTHDAY "ANNUAL REPORT" 
---

More notes here later...

### Import Libraries

In [4]:
# Imports and global options

import pandas as pd
from matplotlib import pyplot as plt
import zipfile
import os
import plotly.express as px 

# display numbers with two decimal places
pd.options.display.float_format = '{:,.2f}'.format

# import one year data
df = pd.read_csv('../../dat/Rilka_20190613_20200111.csv') 

### Clean and Tidy
Notes:
- make 'Date and Time' column into datetime object
- add addition column with City location, based on dates

In [7]:
# clean and tidy the data

df


,Date and Time,End Time,Duration (min),Activity,Quantity,Extra data,Text,Notes,Caregiver
0,1/11/20 11:17,NaN,nan,Diaper,nan,Wet,Rilka Quin Loretta Wilson had a wet diaper (me...,NaN,Daniel
1,1/11/20 10:55,NaN,nan,Bottle,205.00,Milk,Rilka Quin Loretta Wilson drank 205 ml of milk,NaN,Daniel
2,1/11/20 10:55,NaN,nan,Bottle,60.00,Formula,Rilka Quin Loretta Wilson drank 60 ml of formula,NaN,Daniel
3,1/11/20 08:33,1/11/20 09:14,41.00,Sleep,nan,NaN,Rilka Quin Loretta Wilson slept (41m),NaN,Daniel
4,1/11/20 08:31,NaN,nan,Pumping,450.00,ml,450 ml Expressed,NaN,dmv
...,...,...,...,...,...,...,...,...,...
4362,7/16/19 23:24,NaN,nan,Diaper,nan,Poopy+Wet,"Twin A had a poopy and wet diaper (small, normal)",NaN,Daniel
4363,7/16/19 23:24,NaN,nan,Bottle,30.00,Milk,Twin A drank 30 ml of milk,NaN,Daniel
4364,7/13/19 16:11,NaN,nan,Head Size,30.00,cm,Rilka Quin Loretta Wilson's head size: 30 cm,NaN,Daniel
4365,7/13/19 16:10,NaN,nan,Height,42.00,cm,Rilka Quin Loretta Wilson's height: 42 cm,NaN,Daniel


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4367 entries, 0 to 4366
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Date and Time   4367 non-null   object 
 1   End Time        1096 non-null   object 
 2   Duration (min)  1096 non-null   float64
 3   Activity        4367 non-null   object 
 4   Quantity        2332 non-null   float64
 5   Extra data      3279 non-null   object 
 6   Text            4367 non-null   object 
 7   Notes           3 non-null      object 
 8   Caregiver       4367 non-null   object 
dtypes: float64(2), object(7)
memory usage: 307.2+ KB
